In [355]:
import pandas as pd
import numpy as np
import json
import io
import nltk
import re

In [356]:
train_data = []
with io.open('train_data.json','r',encoding='utf8') as f:
    for line in f.readlines():
        d = json.loads(line)
        train_data.append(d)

In [357]:
test_data = []
with io.open('test_data.json','r',encoding='utf8') as f:
    for line in f.readlines():
        d = json.loads(line)
        test_data.append(d)

In [358]:
sentences = []
paragraph_ends = []
i = 0
for paragraph in train_data:
    for sentence in paragraph['Sentences']:
        s = re.findall(r"\w+|[^\w\s]", sentence, re.UNICODE)
        sentences.append(s)
        i += len(s)
    paragraph_ends.append(i)

In [359]:
tokens = []
boundaries = set(paragraph_ends)
offset = 0
for sent in sentences:
    tokens.extend(sent)
    offset += len(sent)
    boundaries.add(offset - 1)

In [395]:
def punct_features(tokens, i):
    t1 = ""
    t2 = ""
    if i + 2 < len(tokens):
        t1 = tokens[i + 2]
    if i - 2 >= 0:
        t2 = tokens[i - 2]
    return {'next-word-capitalized': tokens[i + 1][0].isupper(),
            #'prev-word-capitalized': tokens[i - 1][0].isupper(),
            'prevword': tokens[i - 1],
            'nextword': tokens[i + 1],
#             'is_punct_prev' : tokens[i - 1] in ends,
#             'is_punct_next' : tokens[i + 1] in ends,
            #'is_punct_prev2' : t1 in ends,
            #'is_punct_next2' : t2 in ends,
            'punct': tokens[i],
            'prev-word-is-one-char': len(tokens[i - 1]),
            'next-word-is-one-char': len(tokens[i + 1])
#             'prev-word-is-one-char2': len(t1),
#             'next-word-is-one-char2': len(t2)
           }

In [396]:
ends = [u'!', u'"', u'…', u'.', u'»', u'?']

In [397]:
featuresets = [(punct_features(tokens, i), (i in boundaries))
                   for i in range(len(tokens) - 1)
                   if tokens[i] in ends]
#  and i not in paragraph_ends

In [398]:
print len(featuresets)

91810


In [399]:
out_data = np.zeros((26476, 1))

In [400]:
size = int(len(featuresets) * 0.1)
train_set = featuresets
#train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
#nltk.classify.accuracy(classifier, test_set)

In [401]:
for p in test_data:
    par = p['Paragraph']
    i = 0
    p['Processed'] = []
    for cand in p['Marks']:
        words = re.findall(r"\w+|[^\w\s]", par[i:cand['Pos'] + 1], re.UNICODE)
        p['Processed'].extend(words)
        cand['Word'] = len(p['Processed']) - 1
        i = cand['Pos'] + 1
            
for p in test_data:
    for cand in p['Marks']:
        if cand['Word'] != len(p['Processed']) - 1:
            feature = punct_features(p['Processed'], cand['Word'])
            if classifier.classify(feature) == True:
                out_data[cand['Index'] - 1] = 1
            else:
                out_data[cand['Index'] - 1] = 0
        else:
            out_data[cand['Index'] - 1] = 1

            

In [402]:
df = pd.DataFrame(out_data, columns=['Mark'], index=range(1,26477))
df.index.name = 'Id'

In [403]:
df.to_csv("sampleSubmission.csv")